# Actividad 5 No evaluada Percepton Multicapa

- **Profesor:** Francisco Perez Galarce
- **Ayudante:** Yesenia Salinas

**Integrante:**
- Jorge Troncoso

In [1]:
# Se importan las librerias a utilizar

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# 1.1 Perceptron multicapa

In [2]:
# Preprocesamiento y carga de datos de MNIST
transform = transforms.Compose([
    transforms.ToTensor(),                 # Convertimos imágenes a tensores
    transforms.Normalize((0.5,), (0.5,))  # Normalizamos a media 0 y varianza 1
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Crear dataloaders
batch_size = 64  # Ajusta según la capacidad de GPU/CPU
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

En el siguiente bloque de codigo se hara tanto la modificación de arquitectura como las modifaciones de entrenamiento

In [3]:
# Función para construir un MLP flexible
# Solo se usaran 3 epocas como menciono el profesor para que la ejecución sea mas rapida
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation_function):
        super(MLP, self).__init__()
        layers = []
        current_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(activation_function())
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Aplanar las imágenes
        return self.network(x)

# Función para entrenar el modelo
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

# Función para evaluar el modelo
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Experimentación
results = []
batch_sizes = [32, 64]
learning_rates = [0.001, 0.01]
hidden_layers = [[128], [256, 128], [512, 256, 128]]
activation_functions = [nn.ReLU, nn.Sigmoid]
optimizers = [optim.Adam, optim.SGD]

input_size = 28 * 28  # Tamaño de entrada
output_size = 10  # 10 clases de dígitos

for batch_size in batch_sizes:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    for lr in learning_rates:
        for hidden_sizes in hidden_layers:
            for activation_function in activation_functions:
                for opt_func in optimizers:
                    print(f"Testing: batch_size={batch_size}, lr={lr}, hidden_sizes={hidden_sizes}, activation={activation_function.__name__}, optimizer={opt_func.__name__}")
                    # Crear modelo
                    model = MLP(input_size, hidden_sizes, output_size, activation_function)
                    criterion = nn.CrossEntropyLoss()
                    optimizer = opt_func(model.parameters(), lr=lr)
                    # Entrenar modelo
                    train_model(model, train_loader, optimizer, criterion, num_epochs=3)
                    # Evaluar modelo
                    train_acc = evaluate_model(model, train_loader)
                    test_acc = evaluate_model(model, test_loader)
                    print(f"Train Accuracy: {train_acc * 100:.2f}%, Test Accuracy: {test_acc * 100:.2f}%")
                    # Guardar resultados
                    results.append({
                        'batch_size': batch_size,
                        'learning_rate': lr,
                        'hidden_layers': hidden_sizes,
                        'activation': activation_function.__name__,
                        'optimizer': opt_func.__name__,
                        'train_accuracy': train_acc,
                        'test_accuracy': test_acc
                    })

# Mostrar resultados ordenados por mejor accuracy en test
import pandas as pd
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='test_accuracy', ascending=False)
print(results_df)

Testing: batch_size=32, lr=0.001, hidden_sizes=[128], activation=ReLU, optimizer=Adam
Epoch [1/3], Loss: 0.3447279696742694
Epoch [2/3], Loss: 0.1699446654121081
Epoch [3/3], Loss: 0.12821243075355887
Train Accuracy: 97.03%, Test Accuracy: 96.43%
Testing: batch_size=32, lr=0.001, hidden_sizes=[128], activation=ReLU, optimizer=SGD
Epoch [1/3], Loss: 1.5335675369898478
Epoch [2/3], Loss: 0.7393601686636607
Epoch [3/3], Loss: 0.538245778520902
Train Accuracy: 87.52%, Test Accuracy: 88.01%
Testing: batch_size=32, lr=0.001, hidden_sizes=[128], activation=Sigmoid, optimizer=Adam
Epoch [1/3], Loss: 0.4049938795487086
Epoch [2/3], Loss: 0.1965350634942452
Epoch [3/3], Loss: 0.14589538647284112
Train Accuracy: 96.79%, Test Accuracy: 96.22%
Testing: batch_size=32, lr=0.001, hidden_sizes=[128], activation=Sigmoid, optimizer=SGD
Epoch [1/3], Loss: 2.217266143544515
Epoch [2/3], Loss: 2.0094561297098794
Epoch [3/3], Loss: 1.7560285143534342
Train Accuracy: 71.45%, Test Accuracy: 72.86%
Testing: bat

Epoch [3/3], Loss: 0.11270095981401342
Train Accuracy: 97.17%, Test Accuracy: 96.65%
Testing: batch_size=64, lr=0.001, hidden_sizes=[512, 256, 128], activation=ReLU, optimizer=SGD
Epoch [1/3], Loss: 2.281759505587092
Epoch [2/3], Loss: 2.2175865320762846
Epoch [3/3], Loss: 2.0855780517114506
Train Accuracy: 54.45%, Test Accuracy: 55.26%
Testing: batch_size=64, lr=0.001, hidden_sizes=[512, 256, 128], activation=Sigmoid, optimizer=Adam
Epoch [1/3], Loss: 0.5071390755295849
Epoch [2/3], Loss: 0.1672256277925742
Epoch [3/3], Loss: 0.11762746465283194
Train Accuracy: 97.33%, Test Accuracy: 96.54%
Testing: batch_size=64, lr=0.001, hidden_sizes=[512, 256, 128], activation=Sigmoid, optimizer=SGD
Epoch [1/3], Loss: 2.3065902208214375
Epoch [2/3], Loss: 2.3011423293461424
Epoch [3/3], Loss: 2.301063333493052
Train Accuracy: 11.24%, Test Accuracy: 11.35%
Testing: batch_size=64, lr=0.01, hidden_sizes=[128], activation=ReLU, optimizer=Adam
Epoch [1/3], Loss: 0.4244051267152656
Epoch [2/3], Loss: 0.

## **Informe Detallado de los Experimentos** (Tanto de la arquitectura como del entrenamiento)

## **Objetivo de los Experimentos**
El propósito de estos experimentos fue evaluar cómo diferentes configuraciones arquitectónicas y estrategias de entrenamiento impactan el desempeño de un Perceptrón Multicapa (MLP) entrenado para clasificar dígitos del dataset MNIST. El análisis se llevó a cabo con dos enfoques principales:

1. **Arquitectura del Modelo:**
   - Variar el número de capas ocultas y neuronas.
   - Cambiar la función de activación.
2. **Estrategia de Entrenamiento:**
   - Probar diferentes optimizadores y tasas de aprendizaje.
   - Evaluar el impacto del tamaño del batch.

---

## **Parte 1: Modificaciones en la Arquitectura**

### **Configuraciones Probadas**
Se probaron 3 configuraciones de capas ocultas: 
- `[128]`
- `[256, 128]`
- `[512, 256, 128]`

Cada configuración se probó con dos funciones de activación: 
- `ReLU` (Rectified Linear Unit).
- `Sigmoid`.

### **Resultados Resumidos**

| Arquitectura         | Activación | Optimizador | Train Accuracy | Test Accuracy | Observaciones                                          |
|----------------------|------------|-------------|----------------|---------------|-------------------------------------------------------|
| `[256, 128]`         | `Sigmoid`  | `Adam`      | 97.56%         | **96.98%**    | Excelente generalización con configuraciones intermedias. |
| `[512, 256, 128]`    | `ReLU`     | `Adam`      | 97.28%         | 96.73%        | Profundidad proporciona generalización sólida.         |
| `[128]`              | `ReLU`     | `Adam`      | 97.03%         | 96.43%        | Arquitectura más simple con buen rendimiento.          |
| `[256, 128]`         | `ReLU`     | `Adam`      | 96.81%         | 96.30%        | Buen rendimiento general, menor que configuraciones más profundas. |
| `[128]`              | `Sigmoid`  | `Adam`      | 96.79%         | 96.22%        | `Sigmoid` limita el rendimiento en configuraciones simples. |
| `[512, 256, 128]`    | `Sigmoid`  | `Adam`      | 96.80%         | 96.10%        | Menor precisión comparada con `ReLU`.                  |

### **Análisis y Conclusión**

1. **Función de Activación:**
   - `ReLU` mostró mejor desempeño en general, especialmente en configuraciones profundas.
   - `Sigmoid` tiene problemas con arquitecturas más complejas debido al gradiente desvanecido.

2. **Arquitectura:**
   - Las arquitecturas intermedias (`[256, 128]`) logran el mejor equilibrio entre rendimiento y costo computacional.
   - Redes más profundas como `[512, 256, 128]` ofrecen mayor capacidad de modelado pero requieren más tiempo de entrenamiento.

---

## **Parte 2: Modificaciones en el Entrenamiento**

### **Configuraciones Probadas**
- **Tamaño del batch:** `32` y `64`.
- **Learning rate (lr):** `0.001` y `0.01`.
- **Optimizadores:** `Adam` y `SGD`.

### **Resultados Resumidos**

| Batch Size | LR    | Optimizador | Train Accuracy | Test Accuracy | Observaciones                                    |
|------------|-------|-------------|----------------|---------------|-------------------------------------------------|
| `32`       | 0.001 | `Adam`      | 97.56%         | **96.98%**    | Estabilidad y buena generalización.             |
| `64`       | 0.001 | `Adam`      | 97.33%         | 96.54%        | Precisión sólida con menor costo computacional. |
| `32`       | 0.001 | `SGD`       | 87.52%         | 88.01%        | Lento en converger, menor precisión general.    |
| `32`       | 0.01  | `Adam`      | 92.81%         | 92.74%        | Learning rate alto afecta la generalización.    |
| `64`       | 0.01  | `SGD`       | 91.14%         | 91.50%        | `SGD` mejora con menor batch size.              |

### **Análisis y Conclusión**

1. **Tasa de Aprendizaje:**
   - `lr=0.001` fue óptimo, especialmente con `Adam`.
   - `lr=0.01` mostró menor estabilidad en la generalización.

2. **Optimizador:**
   - `Adam` superó consistentemente a `SGD` en precisión y convergencia.
   - `SGD` es competitivo con tasas de aprendizaje adecuadas pero requiere más ajuste fino.

3. **Tamaño del Batch:**
   - Batch size de `32` proporcionó mejor generalización que `64`, aunque a mayor costo computacional.

---

## **Selección del Modelo Final**

### **Modelo Seleccionado:**
- **Arquitectura:** `[256, 128]` con `Sigmoid`.
- **Entrenamiento:**
  - **Batch size:** `32`.
  - **Learning rate:** `0.001`.
  - **Optimizador:** `Adam`.

### **Rendimiento:**
- **Train Accuracy:** 97.56%.
- **Test Accuracy:** 96.98%.

### **Justificación:**
- La configuración seleccionada equilibra profundidad y estabilidad de entrenamiento.
- Proporciona el mejor desempeño en términos de precisión en pruebas.
- Es computacionalmente eficiente.

---

## **Conclusión General**

1. **Arquitectura:**
   - Las configuraciones intermedias (`[256, 128]`) mostraron ser ideales para este problema, combinando simplicidad y precisión.
   - Configuraciones más simples (`[128]`) son útiles en escenarios con limitaciones computacionales.

2. **Estrategia de Entrenamiento:**
   - El optimizador `Adam` con `lr=0.001` y batch size de `32` fue la mejor combinación para alcanzar alta precisión.

3. **Recomendaciones Futuras:**
   - Extender el número de épocas para confirmar estabilidad en redes profundas.
   - Probar técnicas de regularización como Dropout o Batch Normalization.
   - Aplicar la configuración seleccionada en datasets más complejos para evaluar generalización.

---

## **Apreciaciones Extras**
- **Impacto del Gradiente Desvanecido:** `Sigmoid` funciona bien en arquitecturas simples pero es menos eficiente en configuraciones profundas.
- **Implicaciones Prácticas:** En dispositivos con restricciones, usar configuraciones simples como `[128]` puede ser una solución viable.
